In [4]:
from catboost import CatBoostRanker, Pool
from copy import deepcopy
import numpy as np
import os
import pandas as pd
from collections import Counter


from xgboost import XGBClassifier, XGBRanker
from catboost import CatBoostClassifier, CatBoostRanker
from lightgbm import LGBMClassifier, LGBMRanker
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, RandomForestClassifier
from sklearn.datasets import load_iris 
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, cross_val_score 
from sklearn.metrics import confusion_matrix, classification_report


In [5]:
x, y = make_classification(
    n_samples=5000, 
    n_features= 10, 
    n_classes=3, 
    n_clusters_per_class=1
)

In [6]:
xgbr = XGBRanker(max_depth=5, learning_rate=.01, n_estimators=2000, n_jobs=-1, colsample_bytree=.1)

In [7]:
xgbr.fit(x,y)

ValueError: group or qid is required for ranking task

In [ ]:
from sklearn.model_selection import GroupShuffleSplit

gss = GroupShuffleSplit(test_size=.40, n_splits=1, random_state = 7).split(df, groups=df['id'])

X_train_inds, X_test_inds = next(gss)

train_data= df.iloc[X_train_inds]
X_train = train_data.loc[:, ~train_data.columns.isin(['id','rank'])]
y_train = train_data.loc[:, train_data.columns.isin(['rank'])]

groups = train_data.groupby('id').size().to_frame('size')['size'].to_numpy()

test_data= df.iloc[X_test_inds]

#We need to keep the id for later predictions
X_test = test_data.loc[:, ~test_data.columns.isin(['rank'])]
y_test = test_data.loc[:, test_data.columns.isin(['rank'])]

In [3]:
from catboost.datasets import msrank_10k
train_df, test_df = msrank_10k()

X_train = train_df.drop([0, 1], axis=1).values
y_train = train_df[0].values
queries_train = train_df[1].values

X_test = test_df.drop([0, 1], axis=1).values
y_test = test_df[0].values
queries_test = test_df[1].values

In [4]:
num_documents = X_train.shape[0]
print(num_documents)

10000


In [5]:
X_train.shape[1]


136

In [7]:
Counter(y_train).items()


dict_items([(2.0, 1326), (0.0, 5481), (1.0, 3000), (3.0, 142), (4.0, 51)])

In [8]:
max_relevance = np.max(y_train)
y_train /= max_relevance
y_test /= max_relevance

In [9]:
num_queries = np.unique(queries_train).shape[0]
num_queries

87

In [11]:
train = Pool(
    data=X_train,
    label=y_train,
    group_id=queries_train
)

test = Pool(
    data=X_test,
    label=y_test,
    group_id=queries_test
)

In [12]:
data_dir = './msrank'

if not os.path.exists(data_dir):
    os.makedirs(data_dir)

train_file = os.path.join(data_dir, 'train.csv')
test_file = os.path.join(data_dir, 'test.csv')

train_df.to_csv(train_file, index=False, header=False)
test_df.to_csv(test_file, index=False, header=False)

In [13]:
description_file = os.path.join(data_dir, 'dataset.cd')
with open(description_file, 'w') as f:
    f.write('0\tLabel\n')
    f.write('1\tQueryId\n')

In [14]:
Pool(data=train_file, column_description=description_file, delimiter=',')


In [15]:
default_parameters = {
    'iterations': 2000,
    'custom_metric': ['NDCG', 'PFound', 'AverageGain:top=10'],
    'verbose': False,
    'random_seed': 0,
}

parameters = {}

In [16]:
def fit_model(loss_function, additional_params=None, train_pool=train, test_pool=test):
    parameters = deepcopy(default_parameters)
    parameters['loss_function'] = loss_function
    parameters['train_dir'] = loss_function
    
    if additional_params is not None:
        parameters.update(additional_params)
        
    model = CatBoostRanker(**parameters)
    model.fit(train_pool, eval_set=test_pool, plot=True)
    
    return model

In [18]:
model = fit_model('RMSE', {'custom_metric': ['PrecisionAt:top=10', 'RecallAt:top=10', 'MAP:top=10']})


Custom logger is already specified. Specify more than one logger at same time is not thread safe.

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [19]:
def create_weights(queries):
    query_set = np.unique(queries)
    query_weights = np.random.uniform(size=query_set.shape[0])
    weights = np.zeros(shape=queries.shape)
    
    for i, query_id in enumerate(query_set):
        weights[queries == query_id] = query_weights[i]
    
    return weights
    

train_with_weights = Pool(
    data=X_train,
    label=y_train,
    group_weight=create_weights(queries_train),
    group_id=queries_train
)

test_with_weights = Pool(
    data=X_test,
    label=y_test,
    group_weight=create_weights(queries_test),
    group_id=queries_test
)

fit_model(
    'RMSE', 
    additional_params={'train_dir': 'RMSE_weigths'}, 
    train_pool=train_with_weights,
    test_pool=test_with_weights
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [20]:
def get_best_documents(labels, queries):
    query_set = np.unique(queries)
    num_queries = query_set.shape[0]
    by_query_arg_max = {query: -1 for query in query_set}
    
    for i, query in enumerate(queries):
        best_idx = by_query_arg_max[query]
        if best_idx == -1 or labels[best_idx] < labels[i]:
            by_query_arg_max[query] = i
    
    binary_best_docs = np.zeros(shape=labels.shape)
    for arg_max in by_query_arg_max.values():
        binary_best_docs[arg_max] = 1.
        
    return binary_best_docs

In [21]:
best_docs_train = get_best_documents(y_train, queries_train)
best_docs_test = get_best_documents(y_test, queries_test)

train_with_weights = Pool(
    data=X_train,
    label=best_docs_train,
    group_id=queries_train,
    group_weight=create_weights(queries_train)
)

test_with_weights = Pool(
    data=X_test,
    label=best_docs_test,
    group_id=queries_test,
    group_weight=create_weights(queries_test)
)

fit_model(
    'QuerySoftMax',
    additional_params={'custom_metric': 'AverageGain:top=1'},
    train_pool=train_with_weights,
    test_pool=test_with_weights
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [22]:
fit_model('QueryRMSE')


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [23]:
fit_model('PairLogit')


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [24]:
def read_groups(file_name):
    groups = {}
    group_ids = []

    with open(file_name) as f:
        for doc_id, line in enumerate(f):
            line = line.split(',')[:2]
            
            label, query_id = float(line[0]), int(line[1])
            if query_id not in groups:
                groups[query_id] = []
            groups[query_id].append((doc_id, label))

            group_ids.append(query_id)

    return groups, group_ids
            
train_groups, train_group_ids = read_groups(train_file)
assert num_queries == len(train_groups)

In [25]:
pairs = []

for group in train_groups.values():
    for i in range(len(group)):
        for j in range(i, len(group)):
            if i == j:
                continue
            doc_i, relevance_i = group[i]
            doc_j, relevance_j = group[j]
            if relevance_i < relevance_j:
                pairs.append((doc_j, doc_i))
            else:
                pairs.append((doc_i, doc_j))
                
pairs_file = os.path.join(data_dir, 'pairs.csv')

with open(pairs_file, 'w') as f:
    for pair in pairs:
        f.write(str(pair[0]) + '\t' + str(pair[1]) + '\t1\n')

In [26]:
pool1 = Pool(data=X_train, label=y_train, group_id=train_group_ids, pairs=pairs)
pool2 = Pool(data=train_file, column_description=description_file, pairs=pairs_file, delimiter=',')

In [27]:
fit_model('PairLogitPairwise')


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [ ]:
fit_model('YetiRank')


Training score 0.9999897696469178


In [ ]:
fit_model('YetiRankPairwise')


In [ ]:
widget = MetricVisualizer(['RMSE', 'QueryRMSE', 'PairLogit', 'PairLogitPairwise', 'YetiRank', 'YetiRankPairwise'])
widget.start()

In [ ]:
fit_model('YetiRank', {'metric_period': 50})


In [ ]:
fit_model('YetiRank', {'task_type': 'GPU'})